In [ ]:
# Consolidando arquivos de extração em um único dataframe

import glob
import pandas as pd
import numpy as np

stock_files = glob.glob('/Users/guilhermenomelini/Documents/MBA - Data Science/TCC/Extracao/Data/Extracao_*.csv')
df = pd.concat((pd.read_csv(file, sep=";", encoding="utf-8") for file in stock_files), ignore_index=True)

df.assign(vetor_cbow50 = np.nan, vetor_cbow100 = np.nan, vetor_cbow300 = np.nan, vetor_skip50 = np.nan, vetor_skip100 = np.nan, vetor_skip300 = np.nan, cosine_cbow50_fem = np.nan, cosine_cbow50_masc = np.nan, cosine_cbow100_fem = np.nan, cosine_cbow100_masc = np.nan, cosine_cbow300_fem = np.nan, cosine_cbow300_masc = np.nan, cosine_skip50_fem = np.nan, cosine_skip50_masc = np.nan, cosine_skip100_fem = np.nan, cosine_skip100_masc = np.nan, cosine_skip300_fem = np.nan, cosine_skip300_masc = np.nan)

In [ ]:
# Deletando vagas duplicadas pela chave Descrição + Empresa + Título + Localidade

df = df.drop_duplicates(subset=['Descricao', 'Empresa', 'Titulo', 'Localidade'], keep='first')
df.index = pd.RangeIndex(len(df.index))
df

In [ ]:
# Carregando word embeddings Word2Vec (NILC)

from gensim.models import KeyedVectors

model_cbow50 = KeyedVectors.load_word2vec_format('/Users/guilhermenomelini/Documents/MBA - Data Science/TCC/Modelos/cbow_s50.txt')
model_cbow100 = KeyedVectors.load_word2vec_format('/Users/guilhermenomelini/Documents/MBA - Data Science/TCC/Modelos/cbow_s100.txt')
model_cbow300 = KeyedVectors.load_word2vec_format('/Users/guilhermenomelini/Documents/MBA - Data Science/TCC/Modelos/cbow_s300.txt')
model_skip50 = KeyedVectors.load_word2vec_format('/Users/guilhermenomelini/Documents/MBA - Data Science/TCC/Modelos/skip_s50.txt')
model_skip100 = KeyedVectors.load_word2vec_format('/Users/guilhermenomelini/Documents/MBA - Data Science/TCC/Modelos/skip_s100.txt')
model_skip300 = KeyedVectors.load_word2vec_format('/Users/guilhermenomelini/Documents/MBA - Data Science/TCC/Modelos/skip_s300.txt')

In [ ]:
from nltk import tokenize
from nltk.tokenize import WhitespaceTokenizer
import re

# Definindo caracteres de pontuação para remoção
char_remov = ['!', '(', ')', '[', ']', '{', '}', ';', ',', '"', ':', '/', '.', '<', '>', '?', '@', '#', '$', '%', '^', '&', '*', '_', '~', '•', '|', '``', "''", '+', '°', 'ª']

# Função para deletar emojis
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

# Definindo variável de referência para classe WhitespaceTokenizer
tk = WhitespaceTokenizer()

# Tokenizing
description = []
for i in range(len(df)):
    descricao_tratada = []
    deEmojify(df.Descricao[i])
    tokens = tokenize.word_tokenize(df.Descricao[i], language='portuguese')
    words_lower_case = [w.lower() for w in tokens] #transformando maiúsculas em minúsculas
    filtered_string = [re.sub(r'[0-9]', '', w) for w in words_lower_case] #removendo números
    for j in range(len(filtered_string)):
        for k in filtered_string[j]:
            for char in char_remov: #substituindo os caracteres especiais por espaço
                filtered_string[j] = filtered_string[j].replace(char, " ")
        t = tk.tokenize(filtered_string[j]) #quebrando token em novos tokens tendo espaço como separador
        if len(t) > 0:
            for k in t:
                descricao_tratada.append(k)
    descricao_tratada = [e for e in descricao_tratada if e not in ('h', ' ', '')]
    description.append(descricao_tratada)

In [ ]:
# Definindo função que vetoriza as descrições de emprego

def vetorize(modelo, dimensao):
    vetor_avg = []
    nova_coluna = 'vetor_' + modelo + str(dimensao)
    #CBOW-50
    if modelo == 'cbow' and dimensao == 50:
        for i in range(len(description)): # Loop sobre todas as descrições de emprego
            vetor_descricao = np.empty([len(description[i]), 50])
            vetor_descricao[:] = np.nan
            for j in range(len(description[i])): # Loop para vetorizar cada um dos tokens da descrição de emprego
                if description[i][j] in model_cbow50.key_to_index:
                    vetor_descricao[j] = model_cbow50[description[i][j]].astype('float64')
                else:
                    vetor_descricao[j] = np.empty((1,50,))
            avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0) # Vetorização da descrição de emprego = Cálculo da média dos vetores de todos os tokens da descrição
            # avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
            vetor_avg.append(avg_vetor_descricao)
    #CBOW-100
    elif modelo == 'cbow' and dimensao == 100:
        for i in range(len(description)): # Loop sobre todas as descrições de emprego
            vetor_descricao = np.empty([len(description[i]), 100])
            vetor_descricao[:] = np.nan
            for j in range(len(description[i])): # Loop para vetorizar cada um dos tokens da descrição de emprego
                if description[i][j] in model_cbow100.key_to_index:
                    vetor_descricao[j] = model_cbow100[description[i][j]].astype('float64')
                else:
                    vetor_descricao[j] = np.empty((1,100,))
            avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0) # Vetorização da descrição de emprego = Cálculo da média dos vetores de todos os tokens da descrição
            # avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
            vetor_avg.append(avg_vetor_descricao)
    #CBOW-300
    elif modelo == 'cbow' and dimensao == 300:
        for i in range(len(description)): # Loop sobre todas as descrições de emprego
            vetor_descricao = np.empty([len(description[i]), 300])
            vetor_descricao[:] = np.nan
            for j in range(len(description[i])): # Loop para vetorizar cada um dos tokens da descrição de emprego
                if description[i][j] in model_cbow300.key_to_index:
                    vetor_descricao[j] = model_cbow300[description[i][j]].astype('float64')
                else:
                    vetor_descricao[j] = np.empty((1,300,))
            avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0) # Vetorização da descrição de emprego = Cálculo da média dos vetores de todos os tokens da descrição
            # avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
            vetor_avg.append(avg_vetor_descricao)
    #Skip-gram-50
    elif modelo == 'skip' and dimensao == 50:
        for i in range(len(description)): # Loop sobre todas as descrições de emprego
            vetor_descricao = np.empty([len(description[i]), 50])
            vetor_descricao[:] = np.nan
            for j in range(len(description[i])): # Loop para vetorizar cada um dos tokens da descrição de emprego
                if description[i][j] in model_skip50.key_to_index:
                    vetor_descricao[j] = model_skip50[description[i][j]].astype('float64')
                else:
                    vetor_descricao[j] = np.empty((1,50,))
            avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0) # Vetorização da descrição de emprego = Cálculo da média dos vetores de todos os tokens da descrição
            # avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
            vetor_avg.append(avg_vetor_descricao)
    #Skip-gram-100
    elif modelo == 'skip' and dimensao == 100:
        for i in range(len(description)): # Loop sobre todas as descrições de emprego
            vetor_descricao = np.empty([len(description[i]), 100])
            vetor_descricao[:] = np.nan
            for j in range(len(description[i])): # Loop para vetorizar cada um dos tokens da descrição de emprego
                if description[i][j] in model_skip100.key_to_index:
                    vetor_descricao[j] = model_skip100[description[i][j]].astype('float64')
                else:
                    vetor_descricao[j] = np.empty((1,100,))
            avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0) # Vetorização da descrição de emprego = Cálculo da média dos vetores de todos os tokens da descrição
            # avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
            vetor_avg.append(avg_vetor_descricao)
    #Skip-gram-300
    elif modelo == 'skip' and dimensao == 300:
        for i in range(len(description)): # Loop sobre todas as descrições de emprego
            vetor_descricao = np.empty([len(description[i]), 300])
            vetor_descricao[:] = np.nan
            for j in range(len(description[i])): # Loop para vetorizar cada um dos tokens da descrição de emprego
                if description[i][j] in model_skip300.key_to_index:
                    vetor_descricao[j] = model_skip300[description[i][j]].astype('float64')
                else:
                    vetor_descricao[j] = np.empty((1,300,))
            avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0) # Vetorização da descrição de emprego = Cálculo da média dos vetores de todos os tokens da descrição
            # avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
            vetor_avg.append(avg_vetor_descricao)

    else:
        print("Modelo ou dimensão não permitidos")

    df[nova_coluna] = pd.Series(vetor_avg).values

In [ ]:
# Chamando a função de vetorização para cada arquitetura e dimensionalidade

vetorize('cbow', 50)
vetorize('cbow', 100)
vetorize('cbow', 300)
vetorize('skip', 50)
vetorize('skip', 100)
vetorize('skip', 300)

In [ ]:
df.head()

In [ ]:
# Definindo os vetores de gênero feminino e masculino

# Vetores originais
feminino = ['ela', 'elas', 'filha', 'filhas', 'dela', 'delas', 'mãe', 'mães', 'mulher', 'mulheres', 'garota', 'garotas', 'menina', 'meninas', 'feminino', 'femininos', 'fêmea', 'fêmeas', 'irmã', 'irmãs', 'tia', 'tias', 'sobrinha', 'sobrinhas']
masculino = ['ele', 'eles', 'filho', 'filhos', 'dele', 'deles', 'pai', 'pais', 'homem', 'homens', 'garoto', 'garotos', 'menino', 'meninos', 'masculino', 'masculinos', 'macho', 'machos', 'irmão', 'irmãos', 'tio', 'tios', 'sobrinho', 'sobrinhos']

# Vetores acrescidos de pronomes
# feminino = ['ela', 'elas', 'filha', 'filhas', 'dela', 'delas', 'mãe', 'mães', 'mulher', 'mulheres', 'garota', 'garotas', 'menina', 'meninas', 'feminino', 'femininos', 'fêmea', 'fêmeas', 'irmã', 'irmãs', 'tia', 'tias', 'sobrinha', 'sobrinhas',
#             "nela", "esta", "desta", "nesta", "essa", "dessa", "nessa", "aquela", "daquela", "naquela", "praquela",
#             "nelas", "estas", "destas", "nestas", "essas", "dessas", "nessas", "aquelas", "daquelas", "naquelas", "praquelas"]
# masculino = ['ele', 'eles', 'filho', 'filhos', 'dele', 'deles', 'pai', 'pais', 'homem', 'homens', 'garoto', 'garotos', 'menino', 'meninos', 'masculino', 'masculinos', 'macho', 'machos', 'irmão', 'irmãos', 'tio', 'tios', 'sobrinho', 'sobrinhos',
#             "nele", "este", "deste", "neste", "esse", "desse", "nesse", "aquele", "daquele", "naquele", "praquele",
#             "neles", "estes", "destes", "nestes", "esses", "desses", "nesses", "aqueles", "daqueles", "naqueles", "praqueles"]

# Vetores reduzidos
#feminino = ['ela', 'mulher', 'garota', 'menina', 'feminino', 'fêmea']
#masculino = ['ele', 'homem', 'garoto', 'menino', 'masculino', 'macho']

# Vetores unitários
# feminino = ['mulher']
# masculino = ['homem']


#Criando vetores de gênero feminino

#FEM CBOW50
vetor_genero_fem_cbow50 = []

vetor_descricao = np.empty([len(feminino), 50])
vetor_descricao[:] = np.nan
for j in range(len(feminino)):
    if feminino[j] in model_cbow50.key_to_index:
        vetor_descricao[j] = model_cbow50[feminino[j]].astype('float64')
    else:
        vetor_descricao[j] = np.empty((1,50))
avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
vetor_genero_fem_cbow50.append(avg_vetor_descricao)
vetor_genero_fem_cbow50 = np.array(vetor_genero_fem_cbow50)



#FEM CBOW100
vetor_genero_fem_cbow100 = []

vetor_descricao = np.empty([len(feminino), 100])
vetor_descricao[:] = np.nan
for j in range(len(feminino)):
    if feminino[j] in model_cbow100.key_to_index:
        vetor_descricao[j] = model_cbow100[feminino[j]].astype('float64')
    else:
        vetor_descricao[j] = np.empty((1,100))
avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
vetor_genero_fem_cbow100.append(avg_vetor_descricao)
vetor_genero_fem_cbow100 = np.array(vetor_genero_fem_cbow100)




#FEM CBOW300
vetor_genero_fem_cbow300 = []

vetor_descricao = np.empty([len(feminino), 300])
vetor_descricao[:] = np.nan
for j in range(len(feminino)):
    if feminino[j] in model_cbow300.key_to_index:
        vetor_descricao[j] = model_cbow300[feminino[j]].astype('float64')
    else:
        vetor_descricao[j] = np.empty((1,300))
avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
vetor_genero_fem_cbow300.append(avg_vetor_descricao)
vetor_genero_fem_cbow300 = np.array(vetor_genero_fem_cbow300)



#FEM SKIP50
vetor_genero_fem_skip50 = []

vetor_descricao = np.empty([len(feminino), 50])
vetor_descricao[:] = np.nan
for j in range(len(feminino)):
    if feminino[j] in model_skip50.key_to_index:
        vetor_descricao[j] = model_skip50[feminino[j]].astype('float64')
    else:
        vetor_descricao[j] = np.empty((1,50))
avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
vetor_genero_fem_skip50.append(avg_vetor_descricao)
vetor_genero_fem_skip50 = np.array(vetor_genero_fem_skip50)



#FEM SKIP100
vetor_genero_fem_skip100 = []

vetor_descricao = np.empty([len(feminino), 100])
vetor_descricao[:] = np.nan
for j in range(len(feminino)):
    if feminino[j] in model_skip100.key_to_index:
        vetor_descricao[j] = model_skip100[feminino[j]].astype('float64')
    else:
        vetor_descricao[j] = np.empty((1,100))
avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
vetor_genero_fem_skip100.append(avg_vetor_descricao)
vetor_genero_fem_skip100 = np.array(vetor_genero_fem_skip100)




#FEM SKIP300
vetor_genero_fem_skip300 = []

vetor_descricao = np.empty([len(feminino), 300])
vetor_descricao[:] = np.nan
for j in range(len(feminino)):
    if feminino[j] in model_skip300.key_to_index:
        vetor_descricao[j] = model_skip300[feminino[j]].astype('float64')
    else:
        vetor_descricao[j] = np.empty((1,300))
avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
vetor_genero_fem_skip300.append(avg_vetor_descricao)
vetor_genero_fem_skip300 = np.array(vetor_genero_fem_skip300)











#Criando vetores de gênero masculino

#MASCCBOW50
vetor_genero_masc_cbow50 = []

vetor_descricao = np.empty([len(masculino), 50])
vetor_descricao[:] = np.nan
for j in range(len(masculino)):
    if feminino[j] in model_cbow50.key_to_index:
        vetor_descricao[j] = model_cbow50[masculino[j]].astype('float64')
    else:
        vetor_descricao[j] = np.empty((1,50))
avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
vetor_genero_masc_cbow50.append(avg_vetor_descricao)
vetor_genero_masc_cbow50 = np.array(vetor_genero_masc_cbow50)



#MASC CBOW100
vetor_genero_masc_cbow100 = []

vetor_descricao = np.empty([len(masculino), 100])
vetor_descricao[:] = np.nan
for j in range(len(masculino)):
    if feminino[j] in model_cbow100.key_to_index:
        vetor_descricao[j] = model_cbow100[masculino[j]].astype('float64')
    else:
        vetor_descricao[j] = np.empty((1,100))
avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
vetor_genero_masc_cbow100.append(avg_vetor_descricao)
vetor_genero_masc_cbow100 = np.array(vetor_genero_masc_cbow100)




#MASC CBOW300
vetor_genero_masc_cbow300 = []

vetor_descricao = np.empty([len(masculino), 300])
vetor_descricao[:] = np.nan
for j in range(len(masculino)):
    if feminino[j] in model_cbow300.key_to_index:
        vetor_descricao[j] = model_cbow300[masculino[j]].astype('float64')
    else:
        vetor_descricao[j] = np.empty((1,300))
avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
vetor_genero_masc_cbow300.append(avg_vetor_descricao)
vetor_genero_masc_cbow300 = np.array(vetor_genero_masc_cbow300)



#MASC SKIP50
vetor_genero_masc_skip50 = []

vetor_descricao = np.empty([len(masculino), 50])
vetor_descricao[:] = np.nan
for j in range(len(masculino)):
    if feminino[j] in model_skip50.key_to_index:
        vetor_descricao[j] = model_skip50[masculino[j]].astype('float64')
    else:
        vetor_descricao[j] = np.empty((1,50))
avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
vetor_genero_masc_skip50.append(avg_vetor_descricao)
vetor_genero_masc_skip50 = np.array(vetor_genero_masc_skip50)



#MASC SKIP100
vetor_genero_masc_skip100 = []

vetor_descricao = np.empty([len(masculino), 100])
vetor_descricao[:] = np.nan
for j in range(len(masculino)):
    if feminino[j] in model_skip100.key_to_index:
        vetor_descricao[j] = model_skip100[masculino[j]].astype('float64')
    else:
        vetor_descricao[j] = np.empty((1,100))
avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
vetor_genero_masc_skip100.append(avg_vetor_descricao)
vetor_genero_masc_skip100 = np.array(vetor_genero_masc_skip100)




#MASC SKIP300
vetor_genero_masc_skip300 = []

vetor_descricao = np.empty([len(masculino), 300])
vetor_descricao[:] = np.nan
for j in range(len(masculino)):
    if feminino[j] in model_skip300.key_to_index:
        vetor_descricao[j] = model_skip300[masculino[j]].astype('float64')
    else:
        vetor_descricao[j] = np.empty((1,300))
avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
vetor_genero_masc_skip300.append(avg_vetor_descricao)
vetor_genero_masc_skip300 = np.array(vetor_genero_masc_skip300)

In [ ]:
# Cálculo da similaridade por cosseno

from numpy.linalg import norm


#CBOW50 FEMININO
vetor_fem = []
for i in range(len(df["vetor_cbow50"])):
    similaridade_fem = np.dot(vetor_genero_fem_cbow50, df["vetor_cbow50"][i])/(norm(vetor_genero_fem_cbow50, axis=1)*norm(df["vetor_cbow50"][i]))
    vetor_fem.append(similaridade_fem)
df["cosine_cbow50_fem"] = pd.Series(vetor_fem).values

#CBOW50 MASCULINO
vetor_masc = []
for i in range(len(df["vetor_cbow50"])):
    similaridade_masc = np.dot(vetor_genero_masc_cbow50, df["vetor_cbow50"][i])/(norm(vetor_genero_masc_cbow50, axis=1)*norm(df["vetor_cbow50"][i]))
    vetor_masc.append(similaridade_masc)
df["cosine_cbow50_masc"] = pd.Series(vetor_masc).values



#CBOW100 FEMININO
vetor_fem = []
for i in range(len(df["vetor_cbow100"])):
    similaridade_fem = np.dot(vetor_genero_fem_cbow100, df["vetor_cbow100"][i])/(norm(vetor_genero_fem_cbow100, axis=1)*norm(df["vetor_cbow100"][i]))
    vetor_fem.append(similaridade_fem)
df["cosine_cbow100_fem"] = pd.Series(vetor_fem).values

#CBOW100 MASCULINO
vetor_masc = []
for i in range(len(df["vetor_cbow100"])):
    similaridade_masc = np.dot(vetor_genero_masc_cbow100, df["vetor_cbow100"][i])/(norm(vetor_genero_masc_cbow100, axis=1)*norm(df["vetor_cbow100"][i]))
    vetor_masc.append(similaridade_masc)
df["cosine_cbow100_masc"] = pd.Series(vetor_masc).values



#CBOW300 FEMININO
vetor_fem = []
for i in range(len(df["vetor_cbow300"])):
    similaridade_fem = np.dot(vetor_genero_fem_cbow300, df["vetor_cbow300"][i])/(norm(vetor_genero_fem_cbow300, axis=1)*norm(df["vetor_cbow300"][i]))
    vetor_fem.append(similaridade_fem)
df["cosine_cbow300_fem"] = pd.Series(vetor_fem).values

#CBOW300 MASCULINO
vetor_masc = []
for i in range(len(df["vetor_cbow300"])):
    similaridade_masc = np.dot(vetor_genero_masc_cbow300, df["vetor_cbow300"][i])/(norm(vetor_genero_masc_cbow300, axis=1)*norm(df["vetor_cbow300"][i]))
    vetor_masc.append(similaridade_masc)
df["cosine_cbow300_masc"] = pd.Series(vetor_masc).values



#SKIP50 FEMININO
vetor_fem = []
for i in range(len(df["vetor_skip50"])):
    similaridade_fem = np.dot(vetor_genero_fem_skip50, df["vetor_skip50"][i])/(norm(vetor_genero_fem_skip50, axis=1)*norm(df["vetor_skip50"][i]))
    vetor_fem.append(similaridade_fem)
df["cosine_skip50_fem"] = pd.Series(vetor_fem).values

#SKIP50 MASCULINO
vetor_masc = []
for i in range(len(df["vetor_skip50"])):
    similaridade_masc = np.dot(vetor_genero_masc_skip50, df["vetor_skip50"][i])/(norm(vetor_genero_masc_skip50, axis=1)*norm(df["vetor_skip50"][i]))
    vetor_masc.append(similaridade_masc)
df["cosine_skip50_masc"] = pd.Series(vetor_masc).values



#SKIP100 FEMININO
vetor_fem = []
for i in range(len(df["vetor_skip100"])):
    similaridade_fem = np.dot(vetor_genero_fem_skip100, df["vetor_skip100"][i])/(norm(vetor_genero_fem_skip100, axis=1)*norm(df["vetor_skip100"][i]))
    vetor_fem.append(similaridade_fem)
df["cosine_skip100_fem"] = pd.Series(vetor_fem).values

#SKIP100 MASCULINO
vetor_masc = []
for i in range(len(df["vetor_skip100"])):
    similaridade_masc = np.dot(vetor_genero_masc_skip100, df["vetor_skip100"][i])/(norm(vetor_genero_masc_skip100, axis=1)*norm(df["vetor_skip100"][i]))
    vetor_masc.append(similaridade_masc)
df["cosine_skip100_masc"] = pd.Series(vetor_masc).values



#SKIP300 FEMININO
vetor_fem = []
for i in range(len(df["vetor_skip300"])):
    similaridade_fem = np.dot(vetor_genero_fem_skip300, df["vetor_skip300"][i])/(norm(vetor_genero_fem_skip300, axis=1)*norm(df["vetor_skip300"][i]))
    vetor_fem.append(similaridade_fem)
df["cosine_skip300_fem"] = pd.Series(vetor_fem).values

#SKIP300 MASCULINO
vetor_masc = []
for i in range(len(df["vetor_skip300"])):
    similaridade_masc = np.dot(vetor_genero_masc_skip300, df["vetor_skip300"][i])/(norm(vetor_genero_masc_skip300, axis=1)*norm(df["vetor_skip300"][i]))
    vetor_masc.append(similaridade_masc)
df["cosine_skip300_masc"] = pd.Series(vetor_masc).values

In [ ]:
# Reordenando as colunas

df = df[['Data_Publicacao',
'Descricao',
'Empresa',
'Funcao',
'Localidade',
'Nivel_experiencia',
'Plataforma',
'Setor',
'Tipo_emprego',
'Titulo',
'cosine_cbow50_fem',
'cosine_cbow50_masc',
'cosine_cbow100_fem',
'cosine_cbow100_masc',
'cosine_cbow300_fem',
'cosine_cbow300_masc',
'cosine_skip50_fem',
'cosine_skip50_masc',
'cosine_skip100_fem',
'cosine_skip100_masc',
'cosine_skip300_fem',
'cosine_skip300_masc',
'vetor_cbow50',
'vetor_cbow100',
'vetor_cbow300',
'vetor_skip50',
'vetor_skip100',
'vetor_skip300']]

In [ ]:
df

In [ ]:
### Avaliando consistência entre as arquiteturas (CBOW e Skip-gram)

# Criando coluna de similaridades médias para cada arquitetura

df["cbow_fem_avg"] = df[['cosine_cbow50_fem', 'cosine_cbow100_fem', 'cosine_cbow300_fem']].mean(axis=1)
df["skip_fem_avg"] = df[['cosine_skip50_fem', 'cosine_skip100_fem', 'cosine_skip300_fem']].mean(axis=1)

df["cbow_masc_avg"] = df[['cosine_cbow50_masc', 'cosine_cbow100_masc', 'cosine_cbow300_masc']].mean(axis=1)
df["skip_masc_avg"] = df[['cosine_skip50_masc', 'cosine_skip100_masc', 'cosine_skip300_masc']].mean(axis=1)


# Criando coluna para validar consistência entre arquiteturas: Se CBOW e Skip-gram de determinado gênero apontam para a mesma direção = 1; senão = 0.

df['valida_modelo'] = np.where((df['cbow_fem_avg'] >= df['cbow_masc_avg']) & (df['skip_fem_avg'] >= df['skip_masc_avg'])
                               | (df['cbow_masc_avg'] >= df['cbow_fem_avg']) & (df['skip_masc_avg'] >= df['skip_fem_avg']), 1, 0)
df.head(100)

# df['valida_modelo'].value_counts(normalize=True)
df['valida_modelo'].value_counts(normalize=False)

In [ ]:
### Avaliando equilíbrio entre os gêneros (Fem e Masc)

# Criando condicionais para avaliar gênero predominante (somente para as vagas consistentes entre as arquiteturas)
conditions = [
    df['valida_modelo'] == 0,
    df['valida_modelo'] == 1 & ( df['cbow_fem_avg'] >= df['cbow_masc_avg']) & (df['skip_fem_avg'] >= df['skip_masc_avg']),
    df['valida_modelo'] == 1 & (df['cbow_masc_avg'] >= df['cbow_fem_avg']) & (df['skip_masc_avg'] >= df['skip_fem_avg']),
    ]

values = [0, "Fem", "Masc"]

df['magnitude'] = np.select(conditions, values)
df.head(100)

In [ ]:
mask = df['valida_modelo'] == 1
# df[mask]['magnitude'].value_counts(normalize=True)
df[mask]['magnitude'].value_counts(normalize=False)

In [ ]:
# Análise dos desvios-padrão das similaridades por cosseno entre as dimensionalidades de uma mesma arquitetura e gênero

# Dados para Tabela 3

df['std_cbow_fem'] = np.nan
df['std_cbow_masc'] = np.nan
df['std_skip_fem'] = np.nan
df['std_skip_masc'] = np.nan

for i in range(len(df)):
    df['std_cbow_fem'][i] = np.std([df['cosine_cbow50_fem'][i], df['cosine_cbow100_fem'][i], df['cosine_cbow300_fem'][i]])
    df['std_cbow_masc'][i] = np.std([df['cosine_cbow50_masc'][i], df['cosine_cbow100_masc'][i], df['cosine_cbow300_masc'][i]])
    df['std_skip_fem'][i] = np.std([df['cosine_skip50_fem'][i], df['cosine_skip100_fem'][i], df['cosine_skip300_fem'][i]])
    df['std_skip_masc'][i] = np.std([df['cosine_skip50_masc'][i], df['cosine_skip100_masc'][i], df['cosine_skip300_masc'][i]])

df.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = plt.axes()
ax.scatter(df['cosine_cbow50_fem'], df['cosine_cbow50_masc'], label='50 dimensões', s=0.7, alpha=0.2)
ax.scatter(df['cosine_cbow100_fem'], df['cosine_cbow100_masc'], label='100 dimensões', s=0.7, c='red', alpha=0.2)
ax.scatter(df['cosine_cbow300_fem'], df['cosine_cbow300_masc'], label='300 dimensões', s=0.7, c='orange', alpha=0.2)
ax.set_xlabel("Similaridade Vetor Feminino")
ax.set_ylabel("Similaridade Vetor Masculino")
plt.title('CBOW', loc='center')
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.axvline(x=0, c='black', linewidth=0.5)
ax.axhline(y=0, c='black', linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

lgnd = ax.legend(loc='lower right')
for handle in lgnd.legendHandles:
    handle.set_sizes([30])
    handle.set_alpha(1)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = plt.axes()
ax.scatter(df['cosine_skip50_fem'], df['cosine_skip50_masc'], label='50 dimensões', s=0.7, alpha=0.2)
ax.scatter(df['cosine_skip100_fem'], df['cosine_skip100_masc'], label='100 dimensões', s=0.7, c='red', alpha=0.2)
ax.scatter(df['cosine_skip300_fem'], df['cosine_skip300_masc'], label='300 dimensões', s=0.7, c='orange', alpha=0.2)
ax.set_xlabel("Similaridade Vetor Feminino")
ax.set_ylabel("Similaridade Vetor Masculino")
plt.title('Skip-gram', loc='center')
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.axvline(x=0, c='black', linewidth=0.5)
ax.axhline(y=0, c='black', linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

lgnd = ax.legend(loc='lower right')
for handle in lgnd.legendHandles:
    handle.set_sizes([30])
    handle.set_alpha(1)
plt.show()

# Teste vetores - inclusão pronomes

In [ ]:
df_teste = pd.DataFrame()

palavras_teste_masc = ["ele", "dele", "nele", "este", "deste", "neste", "esse", "nesse", "aquele", "daquele", "naquele", "praquele",
                       "eles", "deles", "neles", "estes", "destes", "nestes", "esses", "nesses", "aqueles", "daqueles", "naqueles", "praqueles"]
palavras_teste_fem = ["ela", "dela", "nela", "esta", "desta", "nesta", "essa", "nessa", "aquela", "daquela", "naquela", "praquela",
                       "elas", "delas", "nelas", "estas", "destas", "nestas", "essas", "nessas", "aquelas", "daquelas", "naquelas", "praquelas"]

df_teste["palavra_masc"] = palavras_teste_masc
df_teste["palavra_fem"] = palavras_teste_fem

vetor_avg = []

for i in range(len(palavras_teste_masc)):
    vetor_descricao = np.empty([len(palavras_teste_masc[i]), 300])
    vetor_descricao[:] = np.nan
    for j in range(len(palavras_teste_masc[i])):
        if description[i][j] in model_cbow300.key_to_index:
            vetor_descricao[j] = model_cbow300[palavras_teste_masc[i][j]].astype('float64')
            # vetor_descricao.append(model[x].astype('float64'))
        else:
            # vetor_descricao.append(np.empty((1,50,)))
            vetor_descricao[j] = np.empty((1,300,))
    avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
    avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
    # print(avg_vetor_descricao)
    # df.vetor[i].append(avg_vetor_descricao)
    vetor_avg.append(avg_vetor_descricao)

df_teste["vetor_masc"] = pd.Series(vetor_avg).values


vetor_avg = []

for i in range(len(palavras_teste_fem)):
    vetor_descricao = np.empty([len(palavras_teste_fem[i]), 300])
    vetor_descricao[:] = np.nan
    for j in range(len(palavras_teste_fem[i])):
        if description[i][j] in model_cbow300.key_to_index:
            vetor_descricao[j] = model_cbow300[palavras_teste_fem[i][j]].astype('float64')
            # vetor_descricao.append(model[x].astype('float64'))
        else:
            # vetor_descricao.append(np.empty((1,50,)))
            vetor_descricao[j] = np.empty((1,300,))
    avg_vetor_descricao = np.nanmean(vetor_descricao.astype('float64'), axis=0)
    avg_vetor_descricao = np.nanmean(vetor_descricao, axis=0)
    # print(avg_vetor_descricao)
    # df.vetor[i].append(avg_vetor_descricao)
    vetor_avg.append(avg_vetor_descricao)

df_teste["vetor_fem"] = pd.Series(vetor_avg).values

In [ ]:
# Criação dos vetores de gênero incluídos de pronomes

df_teste.assign(cosine_cbow300_masc = np.nan, cosine_cbow300_fem = np.nan)

#CBOW300 MASCULINO
vetor_masc= []
for i in range(len(df_teste["vetor_masc"])):
    similaridade_masc = np.dot(vetor_genero_masc_cbow300, df_teste["vetor_masc"][i])/(norm(vetor_genero_masc_cbow300, axis=1)*norm(df_teste["vetor_masc"][i]))
    vetor_masc.append(similaridade_masc)
df_teste["cosine_cbow300_masc"] = pd.Series(vetor_masc).values

#CBOW300 FEMININO
vetor_fem = []
for i in range(len(df_teste["vetor_fem"])):
    similaridade_fem = np.dot(vetor_genero_fem_cbow300, df_teste["vetor_fem"][i])/(norm(vetor_genero_fem_cbow300, axis=1)*norm(df_teste["vetor_fem"][i]))
    vetor_fem.append(similaridade_fem)
df_teste["cosine_cbow300_fem"] = pd.Series(vetor_fem).values

In [ ]:
df_teste

In [ ]:
# Verificando as vagas com as 100 maiores similaridades para cada uma das arquiteturas e gêneros

df2 = df[["Data_Publicacao", "Empresa", "Localidade", "Titulo", "cbow_fem_avg", "skip_fem_avg", "cbow_masc_avg", "skip_masc_avg", "valida_modelo", "magnitude"]]

cbow_h = df2.query("magnitude == 'Masc'").sort_values(by = ['cbow_masc_avg'], ascending=False).head(100)
skip_h = df2.query("magnitude == 'Masc'").sort_values(by = ['skip_masc_avg'], ascending=False).head(100)

cbow_m = df2.query("magnitude == 'Fem'").sort_values(by = ['cbow_fem_avg'], ascending=False).head(100)
skip_m = df2.query("magnitude == 'Fem'").sort_values(by = ['skip_fem_avg'], ascending=False).head(100)

### Testando como o aumento da dimensionalidade afeta a similaridade

In [ ]:
x = "homem"
y = "menino"

dist_cbow50 = np.dot(model_cbow50[y],model_cbow50[x])/(norm(model_cbow50[y])*norm(model_cbow50[x]))
dist_cbow100 = np.dot(model_cbow100[y],model_cbow100[x])/(norm(model_cbow100[y])*norm(model_cbow100[x]))
dist_cbow300 = np.dot(model_cbow300[y],model_cbow300[x])/(norm(model_cbow300[y])*norm(model_cbow300[x]))

dist_skip50 = np.dot(model_skip50[y],model_skip50[x])/(norm(model_skip50[y])*norm(model_skip50[x]))
dist_skip100 = np.dot(model_skip100[y],model_skip100[x])/(norm(model_skip100[y])*norm(model_skip100[x]))
dist_skip300 = np.dot(model_skip300[y],model_skip300[x])/(norm(model_skip300[y])*norm(model_skip300[x]))

print("Similaridade CBOW 50:", dist_cbow50)
print("Similaridade CBOW 100:", dist_cbow100)
print("Similaridade CBOW 300:", dist_cbow300)
print("Similaridade Skip-gram 50:", dist_skip50)
print("Similaridade Skip-gram 100:", dist_skip100)
print("Similaridade Skip-gram 300:", dist_skip300)
print("")
print("Média CBOW:", np.mean([dist_cbow50, dist_cbow100, dist_cbow300]))
print("Média Skip-gram:", np.mean([dist_skip50, dist_skip100, dist_skip300]))
print("Média Combinada", np.mean([dist_cbow50, dist_cbow100, dist_cbow300, dist_skip50, dist_skip100, dist_skip300]))
print("")
print("Desvio-padrão CBOW:", np.std([dist_cbow50, dist_cbow100, dist_cbow300]))
print("Desvio-padrão Skip-gram:", np.std([dist_skip50, dist_skip100, dist_skip300]))
print("Desvio-padrão Combinado", np.std([dist_cbow50, dist_cbow100, dist_cbow300, dist_skip50, dist_skip100, dist_skip300]))


#### Testando a similaridade entre os vetores de gênero e as palavras "homem"/"mulher"

In [ ]:
x = "mulher"

dist_cbow50 = np.dot(vetor_genero_fem_cbow50,model_cbow50[x])/(norm(vetor_genero_fem_cbow50)*norm(model_cbow50[x]))
dist_cbow100 = np.dot(vetor_genero_fem_cbow100,model_cbow100[x])/(norm(vetor_genero_fem_cbow100)*norm(model_cbow100[x]))
dist_cbow300 = np.dot(vetor_genero_fem_cbow300,model_cbow300[x])/(norm(vetor_genero_fem_cbow300)*norm(model_cbow300[x]))

dist_skip50 = np.dot(vetor_genero_fem_skip50,model_skip50[x])/(norm(vetor_genero_fem_skip50)*norm(model_skip50[x]))
dist_skip100 = np.dot(vetor_genero_fem_skip100,model_skip100[x])/(norm(vetor_genero_fem_skip100)*norm(model_skip100[x]))
dist_skip300 = np.dot(vetor_genero_fem_skip300,model_skip300[x])/(norm(vetor_genero_fem_skip300)*norm(model_skip300[x]))

print("Similaridade CBOW 50:", dist_cbow50)
print("Similaridade CBOW 100:", dist_cbow100)
print("Similaridade CBOW 300:", dist_cbow300)
print("Similaridade Skip-gram 50:", dist_skip50)
print("Similaridade Skip-gram 100:", dist_skip100)
print("Similaridade Skip-gram 300:", dist_skip300)
print("")
print("Média CBOW:", np.mean([dist_cbow50, dist_cbow100, dist_cbow300]))
print("Média Skip-gram:", np.mean([dist_skip50, dist_skip100, dist_skip300]))
print("Média Combinada", np.mean([dist_cbow50, dist_cbow100, dist_cbow300, dist_skip50, dist_skip100, dist_skip300]))
print("")
print("Desvio-padrão CBOW:", np.std([dist_cbow50, dist_cbow100, dist_cbow300]))
print("Desvio-padrão Skip-gram:", np.std([dist_skip50, dist_skip100, dist_skip300]))
print("Desvio-padrão Combinado", np.std([dist_cbow50, dist_cbow100, dist_cbow300, dist_skip50, dist_skip100, dist_skip300]))

In [ ]:
x = "homem"

dist_cbow50 = np.dot(vetor_genero_masc_cbow50,model_cbow50[x])/(norm(vetor_genero_masc_cbow50)*norm(model_cbow50[x]))
dist_cbow100 = np.dot(vetor_genero_masc_cbow100,model_cbow100[x])/(norm(vetor_genero_masc_cbow100)*norm(model_cbow100[x]))
dist_cbow300 = np.dot(vetor_genero_masc_cbow300,model_cbow300[x])/(norm(vetor_genero_masc_cbow300)*norm(model_cbow300[x]))

dist_skip50 = np.dot(vetor_genero_masc_skip50,model_skip50[x])/(norm(vetor_genero_masc_skip50)*norm(model_skip50[x]))
dist_skip100 = np.dot(vetor_genero_masc_skip100,model_skip100[x])/(norm(vetor_genero_masc_skip100)*norm(model_skip100[x]))
dist_skip300 = np.dot(vetor_genero_masc_skip300,model_skip300[x])/(norm(vetor_genero_masc_skip300)*norm(model_skip300[x]))

print("Similaridade CBOW 50:", dist_cbow50)
print("Similaridade CBOW 100:", dist_cbow100)
print("Similaridade CBOW 300:", dist_cbow300)
print("Similaridade Skip-gram 50:", dist_skip50)
print("Similaridade Skip-gram 100:", dist_skip100)
print("Similaridade Skip-gram 300:", dist_skip300)
print("")
print("Média CBOW:", np.mean([dist_cbow50, dist_cbow100, dist_cbow300]))
print("Média Skip-gram:", np.mean([dist_skip50, dist_skip100, dist_skip300]))
print("Média Combinada", np.mean([dist_cbow50, dist_cbow100, dist_cbow300, dist_skip50, dist_skip100, dist_skip300]))
print("")
print("Desvio-padrão CBOW:", np.std([dist_cbow50, dist_cbow100, dist_cbow300]))
print("Desvio-padrão Skip-gram:", np.std([dist_skip50, dist_skip100, dist_skip300]))
print("Desvio-padrão Combinado", np.std([dist_cbow50, dist_cbow100, dist_cbow300, dist_skip50, dist_skip100, dist_skip300]))

### Testando se há viés de gênero nos vetores de palavras do NILC

In [ ]:
df_teste_modelo = pd.DataFrame()
df_teste_modelo['palavras'] = model_cbow50.index_to_key

listagem = []

for i in range(len(df_teste_modelo)):
    listagem.insert(i, model_cbow50[df_teste_modelo['palavras'][i]].astype('float64'))

df_teste_modelo['vetor_cbow50'] = listagem

mulher_cbow50 = model_cbow50["mulher"]
homem_cbow50 = model_cbow50["homem"]

df_teste_modelo["cosine_cbow50_fem"] = np.nan
df_teste_modelo["cosine_cbow50_masc"] = np.nan

for i in range(len(df_teste_modelo)):
    df_teste_modelo['cosine_cbow50_fem'][i] = np.dot(mulher_cbow50, df_teste_modelo['vetor_cbow50'][i])/(norm(mulher_cbow50)*norm(df_teste_modelo['vetor_cbow50'][i]))
    df_teste_modelo['cosine_cbow50_masc'][i] = np.dot(homem_cbow50, df_teste_modelo['vetor_cbow50'][i])/(norm(homem_cbow50)*norm(df_teste_modelo['vetor_cbow50'][i]))

In [ ]:
print(len(df_teste_modelo[df_teste_modelo["cosine_cbow50_fem"]>=0])/len(df_teste_modelo))
print(len(df_teste_modelo[df_teste_modelo["cosine_cbow50_masc"]>=0])/len(df_teste_modelo))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# sns.distplot(df_teste_modelo.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_modelo.similaridade_masc, hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_modelo, x="cosine_cbow50_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_modelo, x="cosine_cbow50_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.title('CBOW-50', loc='center')
plt.xlim([-1, 1])

plt.show()

print(df_teste_modelo.describe())

In [ ]:
# Verificando assimetria na distribuição das similaridades

df_teste_modelo.skew(axis = 0, skipna = True)

In [ ]:
# Verificando curtose na distribuição das similaridades

df_teste_modelo.kurtosis(axis = 0, skipna = True)

### Verificando viés nas palavras das descrições

In [ ]:
#Cria dicionário somente com os tokens que são usados nas descrições

i = 0
j = 0
dicionario = []

for i in range(len(df.index)):
    for j in range(len(description[i])):
        dicionario.append(description[i][j])
print(len(dicionario))
dicionario = [*set(dicionario)]
print(len(dicionario))

In [ ]:
df_teste_dicionario = df_teste_modelo
df_teste_dicionario = df_teste_dicionario[df_teste_dicionario['palavras'].isin(dicionario)]

print(len(df_teste_dicionario[df_teste_dicionario["cosine_cbow50_fem"]>=0])/len(df_teste_dicionario))
print(len(df_teste_dicionario[df_teste_dicionario["cosine_cbow50_masc"]>=0])/len(df_teste_dicionario))

# plt.hist(df_teste_dicionario["similaridade_fem"], bins=20, alpha=0.45, color='blue')
# plt.hist(df_teste_dicionario["similaridade_masc"], bins=20, alpha=0.45, color='red')

# sns.distplot(df_teste_dicionario.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_dicionario.similaridade_masc, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_dicionario, x="cosine_cbow50_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_dicionario, x="cosine_cbow50_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.title('CBOW-50', loc='center')
plt.xlim([-1, 1])

plt.show()

In [ ]:
# Verificando assimetria na distribuição das similaridades

df_teste_dicionario.skew(axis = 0, skipna = True)

In [ ]:
# Verificando curtose na distribuição das similaridades

df_teste_dicionario.kurtosis(axis = 0, skipna = True)

In [ ]:
print(dicionario)
print(len(dicionario))
print(len(df_teste_dicionario))

In [ ]:
len(description)

In [ ]:
ax = plt.axes()
ax.scatter(df_teste_dicionario['cosine_cbow50_fem'], df_teste_dicionario['cosine_cbow50_masc'], s=0.7)
ax.set_xlabel("Similaridade 'Mulher'")
ax.set_ylabel("Similaridade 'Homem'")
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.axvline(x=0, c='black', linewidth=0.5)
ax.axhline(y=0, c='black', linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.show()

# Testando se há viés no repositório do NILC (CBOW100)

In [ ]:
df_teste_modelo['palavras_cbow100'] = model_cbow100.index_to_key

listagem = []

for i in range(len(df_teste_modelo)):
    listagem.insert(i, model_cbow100[df_teste_modelo['palavras_cbow100'][i]].astype('float64'))

df_teste_modelo['vetor_cbow100'] = listagem

mulher_cbow100 = model_cbow100["mulher"]
homem_cbow100 = model_cbow100["homem"]

df_teste_modelo["cosine_cbow100_fem"] = np.nan
df_teste_modelo["cosine_cbow100_masc"] = np.nan

for i in range(len(df_teste_modelo)):
    df_teste_modelo['cosine_cbow100_fem'][i] = np.dot(mulher_cbow100, df_teste_modelo['vetor_cbow100'][i])/(norm(mulher_cbow100)*norm(df_teste_modelo['vetor_cbow100'][i]))
    df_teste_modelo['cosine_cbow100_masc'][i] = np.dot(homem_cbow100, df_teste_modelo['vetor_cbow100'][i])/(norm(homem_cbow100)*norm(df_teste_modelo['vetor_cbow100'][i]))

In [ ]:
print(len(df_teste_modelo[df_teste_modelo["cosine_cbow100_fem"]>=0])/len(df_teste_modelo))
print(len(df_teste_modelo[df_teste_modelo["cosine_cbow100_masc"]>=0])/len(df_teste_modelo))

In [ ]:
# sns.distplot(df_teste_modelo.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_modelo.similaridade_masc, hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_modelo, x="cosine_cbow100_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_modelo, x="cosine_cbow100_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.title('CBOW-100', loc='center')
plt.xlim([-1, 1])

plt.show()

In [ ]:
# sns.distplot(df_teste_modelo.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_modelo.similaridade_masc, hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_modelo, x="cosine_cbow50_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_modelo, x="cosine_cbow50_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.xlim([-1, 1])

plt.show()


In [ ]:
df_teste_dicionario = df_teste_modelo
df_teste_dicionario = df_teste_dicionario[df_teste_dicionario['palavras'].isin(dicionario)]

print(len(df_teste_dicionario[df_teste_dicionario["cosine_cbow100_fem"]>=0])/len(df_teste_dicionario))
print(len(df_teste_dicionario[df_teste_dicionario["cosine_cbow100_masc"]>=0])/len(df_teste_dicionario))

# plt.hist(df_teste_dicionario["similaridade_fem"], bins=20, alpha=0.45, color='blue')
# plt.hist(df_teste_dicionario["similaridade_masc"], bins=20, alpha=0.45, color='red')

# sns.distplot(df_teste_dicionario.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_dicionario.similaridade_masc, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_dicionario, x="cosine_cbow100_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_dicionario, x="cosine_cbow100_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.title('CBOW-100', loc='center')
plt.xlim([-1, 1])

plt.show()


In [ ]:
ax = plt.axes()
ax.scatter(df_teste_dicionario['cosine_cbow50_fem'], df_teste_dicionario['cosine_cbow50_masc'], s=0.7, alpha=0.2)
ax.scatter(df_teste_dicionario['cosine_cbow100_fem'], df_teste_dicionario['cosine_cbow100_masc'], s=0.7, c='red', alpha=0.2)
ax.set_xlabel("Similaridade 'Mulher'")
ax.set_ylabel("Similaridade 'Homem'")
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.axvline(x=0, c='black', linewidth=0.5)
ax.axhline(y=0, c='black', linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.show()

# Testando se há viés no repositório do NILC (CBOW 300)

In [ ]:
df_teste_modelo['palavras'] = model_cbow300.index_to_key

listagem = []

for i in range(len(df_teste_modelo)):
    listagem.insert(i, model_cbow300[df_teste_modelo['palavras'][i]].astype('float64'))

df_teste_modelo['vetor_cbow300'] = listagem

mulher_cbow300 = model_cbow300["mulher"]
homem_cbow300 = model_cbow300["homem"]

df_teste_modelo["cosine_cbow300_fem"] = np.nan
df_teste_modelo["cosine_cbow300_masc"] = np.nan

for i in range(len(df_teste_modelo)):
    df_teste_modelo['cosine_cbow300_fem'][i] = np.dot(mulher_cbow300, df_teste_modelo['vetor_cbow300'][i])/(norm(mulher_cbow300)*norm(df_teste_modelo['vetor_cbow300'][i]))
    df_teste_modelo['cosine_cbow300_masc'][i] = np.dot(homem_cbow300, df_teste_modelo['vetor_cbow300'][i])/(norm(homem_cbow300)*norm(df_teste_modelo['vetor_cbow300'][i]))

In [ ]:
print(len(df_teste_modelo[df_teste_modelo["cosine_cbow300_fem"]>=0])/len(df_teste_modelo))
print(len(df_teste_modelo[df_teste_modelo["cosine_cbow300_masc"]>=0])/len(df_teste_modelo))

In [ ]:
# sns.distplot(df_teste_modelo.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_modelo.similaridade_masc, hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_modelo, x="cosine_cbow300_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_modelo, x="cosine_cbow300_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.title('CBOW-300', loc='center')
plt.xlim([-1, 1])

plt.show()

In [ ]:
df_teste_dicionario = df_teste_modelo
df_teste_dicionario = df_teste_dicionario[df_teste_dicionario['palavras'].isin(dicionario)]

print(len(df_teste_dicionario[df_teste_dicionario["cosine_cbow300_fem"]>=0])/len(df_teste_dicionario))
print(len(df_teste_dicionario[df_teste_dicionario["cosine_cbow300_masc"]>=0])/len(df_teste_dicionario))

# plt.hist(df_teste_dicionario["similaridade_fem"], bins=20, alpha=0.45, color='blue')
# plt.hist(df_teste_dicionario["similaridade_masc"], bins=20, alpha=0.45, color='red')

# sns.distplot(df_teste_dicionario.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_dicionario.similaridade_masc, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_dicionario, x="cosine_cbow300_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_dicionario, x="cosine_cbow300_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.title('CBOW-300', loc='center')
plt.xlim([-1, 1])

plt.show()

In [ ]:
ax = plt.axes()
ax.scatter(df_teste_dicionario['cosine_cbow50_fem'], df_teste_dicionario['cosine_cbow50_masc'], label='CBOW-50', s=0.7, alpha=0.2)
ax.scatter(df_teste_dicionario['cosine_cbow100_fem'], df_teste_dicionario['cosine_cbow100_masc'], label='CBOW-100', s=0.7, c='red', alpha=0.2)
ax.scatter(df_teste_dicionario['cosine_cbow300_fem'], df_teste_dicionario['cosine_cbow300_masc'], label='CBOW-300', s=0.7, c='orange', alpha=0.2)
ax.set_xlabel("Similaridade 'Mulher'")
ax.set_ylabel("Similaridade 'Homem'")
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.axvline(x=0, c='black', linewidth=0.5)
ax.axhline(y=0, c='black', linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

lgnd = ax.legend(loc='lower right')
for handle in lgnd.legendHandles:
    handle.set_sizes([30])
    handle.set_alpha(1)
plt.show()

# Testando se há viés no repositório do NILC e no dicionário (Skip-gram)

## 50 Dimensões

### NILC

In [ ]:
df_teste_modelo['palavras'] = model_skip50.index_to_key

listagem = []

for i in range(len(df_teste_modelo)):
    listagem.insert(i, model_skip50[df_teste_modelo['palavras'][i]].astype('float64'))

df_teste_modelo['vetor_skip50'] = listagem

mulher_skip50 = model_skip50["mulher"]
homem_skip50 = model_skip50["homem"]

df_teste_modelo["cosine_skip50_fem"] = np.nan
df_teste_modelo["cosine_skip50_masc"] = np.nan

for i in range(len(df_teste_modelo)):
    df_teste_modelo['cosine_skip50_fem'][i] = np.dot(mulher_skip50, df_teste_modelo['vetor_skip50'][i])/(norm(mulher_skip50)*norm(df_teste_modelo['vetor_skip50'][i]))
    df_teste_modelo['cosine_skip50_masc'][i] = np.dot(homem_skip50, df_teste_modelo['vetor_skip50'][i])/(norm(homem_skip50)*norm(df_teste_modelo['vetor_skip50'][i]))

print(len(df_teste_modelo[df_teste_modelo["cosine_skip50_fem"]>=0])/len(df_teste_modelo))
print(len(df_teste_modelo[df_teste_modelo["cosine_skip50_masc"]>=0])/len(df_teste_modelo))

In [ ]:
print(len(df_teste_modelo[df_teste_modelo["cosine_skip50_fem"]>=0])/len(df_teste_modelo))
print(len(df_teste_modelo[df_teste_modelo["cosine_skip50_masc"]>=0])/len(df_teste_modelo))

In [ ]:
# sns.distplot(df_teste_modelo.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_modelo.similaridade_masc, hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_modelo, x="cosine_skip50_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_modelo, x="cosine_skip50_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.title('Skip-gram-50', loc='center')
plt.xlim([-1, 1])

plt.show()

### Dicionário

In [ ]:
df_teste_dicionario = df_teste_modelo
df_teste_dicionario = df_teste_dicionario[df_teste_dicionario['palavras'].isin(dicionario)]

print(len(df_teste_dicionario[df_teste_dicionario["cosine_skip50_fem"]>=0])/len(df_teste_dicionario))
print(len(df_teste_dicionario[df_teste_dicionario["cosine_skip50_masc"]>=0])/len(df_teste_dicionario))

# plt.hist(df_teste_dicionario["similaridade_fem"], bins=20, alpha=0.45, color='blue')
# plt.hist(df_teste_dicionario["similaridade_masc"], bins=20, alpha=0.45, color='red')

# sns.distplot(df_teste_dicionario.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_dicionario.similaridade_masc, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_dicionario, x="cosine_skip50_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_dicionario, x="cosine_skip50_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.title('Skip-gram-50', loc='center')
plt.xlim([-1, 1])

plt.show()

## 100 Dimensões

### NILC

In [ ]:
df_teste_modelo['palavras'] = model_skip100.index_to_key

listagem = []

for i in range(len(df_teste_modelo)):
    listagem.insert(i, model_skip100[df_teste_modelo['palavras'][i]].astype('float64'))

df_teste_modelo['vetor_skip100'] = listagem

mulher_skip100 = model_skip100["mulher"]
homem_skip100 = model_skip100["homem"]

df_teste_modelo["cosine_skip100_fem"] = np.nan
df_teste_modelo["cosine_skip100_masc"] = np.nan

for i in range(len(df_teste_modelo)):
    df_teste_modelo['cosine_skip100_fem'][i] = np.dot(mulher_skip100, df_teste_modelo['vetor_skip100'][i])/(norm(mulher_skip100)*norm(df_teste_modelo['vetor_skip100'][i]))
    df_teste_modelo['cosine_skip100_masc'][i] = np.dot(homem_skip100, df_teste_modelo['vetor_skip100'][i])/(norm(homem_skip100)*norm(df_teste_modelo['vetor_skip100'][i]))

print(len(df_teste_modelo[df_teste_modelo["cosine_skip100_fem"]>=0])/len(df_teste_modelo))
print(len(df_teste_modelo[df_teste_modelo["cosine_skip100_masc"]>=0])/len(df_teste_modelo))

In [ ]:
print(len(df_teste_modelo[df_teste_modelo["cosine_skip100_fem"]>=0])/len(df_teste_modelo))
print(len(df_teste_modelo[df_teste_modelo["cosine_skip100_masc"]>=0])/len(df_teste_modelo))

In [ ]:
# sns.distplot(df_teste_modelo.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_modelo.similaridade_masc, hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_modelo, x="cosine_skip100_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_modelo, x="cosine_skip100_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.title('Skip-gram-100', loc='center')
plt.xlim([-1, 1])

plt.show()

### Dicionário

In [ ]:
df_teste_dicionario = df_teste_modelo
df_teste_dicionario = df_teste_dicionario[df_teste_dicionario['palavras'].isin(dicionario)]

print(len(df_teste_dicionario[df_teste_dicionario["cosine_skip100_fem"]>=0])/len(df_teste_dicionario))
print(len(df_teste_dicionario[df_teste_dicionario["cosine_skip100_masc"]>=0])/len(df_teste_dicionario))

# plt.hist(df_teste_dicionario["similaridade_fem"], bins=20, alpha=0.45, color='blue')
# plt.hist(df_teste_dicionario["similaridade_masc"], bins=20, alpha=0.45, color='red')

# sns.distplot(df_teste_dicionario.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_dicionario.similaridade_masc, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_dicionario, x="cosine_skip100_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_dicionario, x="cosine_skip100_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.title('Skip-gram-100', loc='center')
plt.xlim([-1, 1])

plt.show()

## 300 Dimensões

### NILC

In [ ]:
df_teste_modelo['palavras'] = model_skip300.index_to_key

listagem = []

for i in range(len(df_teste_modelo)):
    listagem.insert(i, model_skip300[df_teste_modelo['palavras'][i]].astype('float64'))

df_teste_modelo['vetor_skip300'] = listagem

mulher_skip300 = model_skip300["mulher"]
homem_skip300 = model_skip300["homem"]

df_teste_modelo["cosine_skip300_fem"] = np.nan
df_teste_modelo["cosine_skip300_masc"] = np.nan

for i in range(len(df_teste_modelo)):
    df_teste_modelo['cosine_skip300_fem'][i] = np.dot(mulher_skip300, df_teste_modelo['vetor_skip300'][i])/(norm(mulher_skip300)*norm(df_teste_modelo['vetor_skip300'][i]))
    df_teste_modelo['cosine_skip300_masc'][i] = np.dot(homem_skip300, df_teste_modelo['vetor_skip300'][i])/(norm(homem_skip300)*norm(df_teste_modelo['vetor_skip300'][i]))

print(len(df_teste_modelo[df_teste_modelo["cosine_skip300_fem"]>=0])/len(df_teste_modelo))
print(len(df_teste_modelo[df_teste_modelo["cosine_skip300_masc"]>=0])/len(df_teste_modelo))

In [ ]:
print(len(df_teste_modelo[df_teste_modelo["cosine_skip300_fem"]>=0])/len(df_teste_modelo))
print(len(df_teste_modelo[df_teste_modelo["cosine_skip300_masc"]>=0])/len(df_teste_modelo))

In [ ]:
# sns.distplot(df_teste_modelo.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_modelo.similaridade_masc, hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_modelo, x="cosine_skip300_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_modelo, x="cosine_skip300_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.title('Skip-gram-300', loc='center')
plt.xlim([-1, 1])

plt.show()

print(df_teste_modelo.describe())

### Dicionário

In [ ]:
df_teste_dicionario = df_teste_modelo
df_teste_dicionario = df_teste_dicionario[df_teste_dicionario['palavras'].isin(dicionario)]

print(len(df_teste_dicionario[df_teste_dicionario["cosine_skip300_fem"]>=0])/len(df_teste_dicionario))
print(len(df_teste_dicionario[df_teste_dicionario["cosine_skip300_masc"]>=0])/len(df_teste_dicionario))

# plt.hist(df_teste_dicionario["similaridade_fem"], bins=20, alpha=0.45, color='blue')
# plt.hist(df_teste_dicionario["similaridade_masc"], bins=20, alpha=0.45, color='red')

# sns.distplot(df_teste_dicionario.similaridade_fem, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))
# sns.distplot(df_teste_dicionario.similaridade_masc, axlabel = 'Similaridade', hist_kws=dict(alpha=0.3))

sns.histplot(data=df_teste_dicionario, x="cosine_skip300_fem", kde=True, color="lightskyblue", alpha=0.4)
sns.histplot(data=df_teste_dicionario, x="cosine_skip300_masc", kde=True, color="orange", alpha=0.4)

plt.legend(['"Mulher"', '"Homem"'])
plt.xlabel('Similaridade')
plt.ylabel('Frequência')
plt.title('Skip-gram-300', loc='center')
plt.xlim([-1, 1])

plt.show()

print(df_teste_dicionario.describe())

### Comparativo Dimensões

In [ ]:
ax = plt.axes()
ax.scatter(df_teste_dicionario['cosine_skip50_fem'], df_teste_dicionario['cosine_skip50_masc'], label='Skip-gram-50', s=0.7, alpha=0.2)
ax.scatter(df_teste_dicionario['cosine_skip100_fem'], df_teste_dicionario['cosine_skip100_masc'], label='Skip-gram-100', s=0.7, c='red', alpha=0.2)
ax.scatter(df_teste_dicionario['cosine_skip300_fem'], df_teste_dicionario['cosine_skip300_masc'], label='Skip-gram-300', s=0.7, c='orange', alpha=0.2)
ax.set_xlabel("Similaridade 'Mulher'")
ax.set_ylabel("Similaridade 'Homem'")
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.axvline(x=0, c='black', linewidth=0.5)
ax.axhline(y=0, c='black', linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

lgnd = ax.legend(loc='lower right')
for handle in lgnd.legendHandles:
    handle.set_sizes([30])
    handle.set_alpha(1)
plt.show()